<a href="https://colab.research.google.com/github/Ricardojnf33/sistema_de_recomendacao/blob/main/product_recommendation_apriori_causal_mlV2_SCRIPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🎖Product Recommendation - Apriori & causal ML - Link:
https://www.kaggle.com/code/hunter0007/product-recommendation-apriori-causal-ml

## ENTENDIMENTO DE NEGÓCIOS

Hunter's e-grocery é uma conhecida marca francesa com sede na Bretanha que oferece uma nova geração de produtos de e-mercearia e estilo de vida. Com presença em dez países, o e-grocery da Hunter busca continuamente novas maneiras de antecipar e atender às demandas dos clientes, construindo uma forte lealdade entre seus clientes. No entanto, eventos recentes de cisnes negros, como o Covid-19, a crise na Ucrânia e a escassez de gás, impactaram o comportamento de compra, tornando necessário desenvolver uma proposta de valores de negócios para aumentar o valor da vida útil do cliente. Para garantir a conclusão bem-sucedida deste extenso estudo, implementaremos uma metodologia de 5 etapas.

<a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
<p style="background-color:#be6f37;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">ÍNDICE</p>

[VISÃO GERAL](#0)
- [1. COMPREENDENDO O NEGÓCIO](#1)

- [2. COMPREENDENDO OS DADOS](#2)

    - [2.1 Obter informações sobre os dados](#2.1)
    - [2.2 Realizando estatísticas resumidas](#2.2)
    - [2.3 Lidando com valores nulos](#2.3)
    - [2.4 Convertendo tipos de dados](#2.4)
- [3. PREPARAÇÃO DOS DADOS](#3)
    - [3.1 Investigando todos os elementos de cada atributo](#3.1)
    - [3.2 Agregando e agrupando valores para visualizar o comportamento de compra](#3.2)
    - [3.3 Criando atributos baseados no tempo](#3.3)
    - [3.4 Estudo de heatmap](#3.4)
    - [3.5 Visualizando o período de tempo do pedido](#3.5)
    - [3.6 Criando atributo de grupo de pedido](#3.6)
    - [3.7 Visualizando o atributo de grupo de pedido](#3.7)
    - [3.8 Visualizando os 10 principais produtos](#3.8)
    - [3.9 Visualizando os 10 produtos com menor quantidade](#3.9)
    - [3.9 Agregando e agrupando valores para entender o comportamento de compra](#3.10)
    - [3.10 Agregando e agrupando valores para entender a taxa de reincidência do produto](#3.11)
    - [3.11 Agregando e agrupando valores para visualizar a taxa de reincidência](#3.12)

- [4. MODELAGEM](#4)

    - [4.1 Preparando e construindo o algoritmo apriori](#4.1)
    - [4.2 Construindo parâmetros personalizados para o gráfico](#4.2)
    - [4.3 BConstruindo os itens de ocorrência frequente](#4.3)
    - [4.4 Adicionando contagem de conjuntos de itens à lista de itens frequentes](#4.4)

- [5. IMPLANTAÇÃO](#5)
    - [5.1 Modelos de Inferência Causal e Estratégia de Relacionamentos](#5.1)
- [6. CONCLUSÃO](#6)

In [43]:
# IMPORTING PACKAGES
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.io as pio
import seaborn as sns
from datetime import datetime
import sklearn
import plotly.express as px
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.cluster import KMeans , k_means
from IPython.display import Image, display, HTML
import warnings
warnings.filterwarnings('ignore')

In [46]:
!wget "https://www.kaggle.com/datasets/hunter0007/ecommerce-dataset-for-predictive-marketing-2023/download?datasetVersionNumber=1"

--2023-07-12 18:02:33--  https://www.kaggle.com/datasets/hunter0007/ecommerce-dataset-for-predictive-marketing-2023/download?datasetVersionNumber=1
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Fhunter0007%2Fecommerce-dataset-for-predictive-marketing-2023%2Fversions%2F1%3Fresource%3Ddownload [following]
--2023-07-12 18:02:33--  https://www.kaggle.com/account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Fhunter0007%2Fecommerce-dataset-for-predictive-marketing-2023%2Fversions%2F1%3Fresource%3Ddownload
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘download?da

In [44]:
main_data= pd.read_csv("/content/ECommerce_consumer behaviour.csv")
df = main_data.copy()
df

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,2425083,49125,1,2,18,NaN,17,1,0,13,pantry,baking ingredients
1,2425083,49125,1,2,18,NaN,91,2,0,16,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,NaN,36,3,0,16,dairy eggs,butter
3,2425083,49125,1,2,18,NaN,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables
...,...,...,...,...,...,...,...,...,...,...,...,...
2019496,3390742,199430,16,3,18,5.0,83,8,0,4,produce,fresh vegetables
2019497,458285,128787,42,2,19,3.0,115,1,1,7,beverages,water seltzer sparkling water
2019498,458285,128787,42,2,19,3.0,32,2,1,4,produce,packaged produce
2019499,458285,128787,42,2,19,3.0,32,3,1,4,produce,packaged produce


<a id="2"></a>  
<p style="background-color:#be6f37;font-family:newtimeroman;color:#FFF9ED;font-size:120%;text-align:center;border-radius:10px 10px;"> ENTENDIMENTO DOS DADOS</p>

#### **Os dados consistem em 2019501 linhas e 12 colunas, que são as seguintes:
**

- order_id – (Um número único para identificar o pedido)
- user_id - (Um número único para identificar o usuário)
- order_number – (Número do pedido)
- order_dow – (Dia da semana em que o pedido foi feito)
- order_hour_of_day – (Hora do pedido)
- days_since_prior_order - (Histórico do pedido)
- product_id – (ID do produto)
- add_to_cart_order – (Número de itens adicionados ao carrinho)
- reordered – (Se houve reincidência do pedido)
- department_id - (Número único atribuído a cada departamento)
- department – (Nomes dos departamentos)
- product_name – (Nome dos produtos)

<a id="2.1"></a>
## <b>2.1<span style='color:#be6f37'> Obtenha informações sobre os dados</span></b>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder

# Carregar o dataframe
df = pd.read_csv("/content/ECommerce_consumer behaviour.csv")

# Informações sobre o dataframe
df_info = df.info()

# Tratar valores nulos
df.fillna(-1, inplace=True)

# Converter tipos de dados
df['days_since_prior_order'] = df['days_since_prior_order'].astype(np.int64)

# Análise exploratória de cada coluna
def explore_column(col):
    unique_values = df[col].unique()
    num_values = len(unique_values)
    if num_values < 22:
        values = df[col].value_counts().reset_index()
        values.columns = [col, 'Count']
        print(f"The number of unique values for column '{col}': {num_values}")
        print(values)
    else:
        print(f"The number of unique values for column '{col}': {num_values}")

# Explorar cada coluna do dataframe
for col in df.columns:
    explore_column(col)

# Visualizar o comportamento de compra - Número de produtos adicionados ao carrinho
grouped_cart = df.groupby("order_id")["add_to_cart_order"].max().value_counts().reset_index()
grouped_cart.columns = ["Number of Products Added to Cart", "Count"]

plt.figure(figsize=(15, 12))
sns.barplot(x="Number of Products Added to Cart", y="Count", data=grouped_cart, palette="rocket_r")
plt.xlabel("Number of Products Added to Cart", fontsize=15)
plt.ylabel("Number of Unique Orders", fontsize=15)
plt.title("Purchasing Behavior by Number of Products Added to Cart", fontsize=18)
plt.xlim(0, 35)
plt.show()

# Visualizar o comportamento de compra - Horário do dia
grouped_hour = df.groupby('order_hour_of_day')['user_id'].count().reset_index()

plt.figure(figsize=(15, 12))
sns.barplot(x='order_hour_of_day', y='user_id', data=grouped_hour)
plt.xlabel("Time of the day", fontsize=13)
plt.ylabel("Number of Unique Orders", fontsize=13)
plt.title("Purchasing Behavior by Time of the Day", fontsize=18)
plt.xticks(rotation='vertical')
plt.show()

# Heatmap - Dias da semana e horas do dia
day_hour_df = df.groupby(["order_dow", "order_hour_of_day"])["order_number"].count().reset_index()
day_hour_pivot = day_hour_df.pivot('order_dow', 'order_hour_of_day', 'order_number') / df.shape[0]

plt.figure(figsize=(15, 10))
sns.heatmap(day_hour_pivot, cmap='YlGnBu')
plt.yticks([0, 1, 2, 3, 4, 5, 6], ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
plt.xlabel("Hour of day")
plt.ylabel("Day of week")
plt.title("Crosstable of Days and Hours of Orders")
plt.show()

# Criar coluna para período do dia do pedido
def get_order_time_period(hour):
    if 6 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 18:
        return "Afternoon"
    elif 18 <= hour < 23:
        return "Evening"
    else:
        return "Night"

df['order_time_period'] = df['order_hour_of_day'].apply(get_order_time_period)

# Visualizar o período de tempo do pedido
orders_by_day_time = df.pivot_table(index='order_dow', columns='order_time_period', values='user_id', aggfunc='count')

plt.figure(figsize=(15, 15))
ax = orders_by_day_time.plot(kind='bar', color=sns.color_palette("rocket_r", len(orders_by_day_time.columns)))
ax.set_title('Time when Orders were Made by Day and Time of Day', fontsize=20)
ax.set_xlabel('Day of Week', fontsize=15)
ax.set_ylabel('Number of Orders', fontsize=15)
ax.set_xticklabels(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
ax.legend(title='Order Time Period', fontsize=12)
plt.show()

# Criar grupo de pedidos
def get_order_number_group(num_orders):
    ranges = [(1, 10), (11, 20), (21, 30), (31, 40), (41, 50), (51, 60), (61, 70), (71, 80), (81, 90), (91, 100)]
    for r in ranges:
        if r[0] <= num_orders <= r[1]:
            return f"{r[0]}-{r[1]} orders"
    return "More than 100 orders"

df['order_number_group'] = df['order_number'].apply(get_order_number_group)

# Visualizar o grupo de pedidos
orders_by_group = df['order_number_group'].value_counts().sort_values(ascending=False)

plt.figure(figsize=(15, 15))
plt.pie(orders_by_group, labels=orders_by_group.index, autopct='%1.2f%%')
plt.title('Number of Orders by Order Number Group', fontsize=18)
plt.show()

# Visualizar os principais produtos
top_products = df['product_name'].value_counts().head(10)

plt.figure(figsize=(15, 15))
sns.barplot(x=top_products.values, y=top_products.index, palette="rocket_r")
plt.xlabel("Number of Orders", fontsize=13)
plt.ylabel("Product Name", fontsize=13)
plt.title("Top 10 Products", fontsize=18)
plt.show()

# Visualizar os últimos produtos
bottom_products = df['product_name'].value_counts().tail(10)

plt.figure(figsize=(15, 15))
sns.barplot(x=bottom_products.values, y=bottom_products.index, palette="rocket_r")
plt.xlabel("Number of Orders", fontsize=13)
plt.ylabel("Product Name", fontsize=13)
plt.title("Bottom 10 Products", fontsize=18)
plt.show()

# Agrupar valores para entender o comportamento de compra por departamento
grouped_department = df.groupby(["department", "order_time_period"])["reordered"].count().reset_index()
grouped_department = grouped_department.pivot('department', 'order_time_period', 'reordered').fillna(0)

plt.figure(figsize=(15, 15))
sns.heatmap(grouped_department, cmap='YlGnBu', annot=True, fmt='g')
plt.xlabel("Order Time Period", fontsize=13)
plt.ylabel("Department", fontsize=13)
plt.title("Purchasing Behavior by Department and Order Time Period", fontsize=18)
plt.show()

# Taxa de reordenação dos produtos
reordered_count = df['reordered'].value_counts()
reordered_ratio = reordered_count / reordered_count.sum() * 100

plt.figure(figsize=(6, 8))
sns.barplot(x=reordered_ratio.index, y=reordered_ratio.values, palette="coolwarm")
plt.xlabel("Reordered", fontsize=13)
plt.ylabel("Percentage", fontsize=13)
plt.title("Reordered Product Ratio", fontsize=18)
plt.ylim(0, 100)
plt.show()

# Aplicar o algoritmo Apriori para encontrar conjuntos de itens frequentes
transactions = df.groupby(['user_id', 'department'])['product_name'].apply(list).reset_index(name='transactions')
te = TransactionEncoder()
te_ary = te.fit_transform(transactions['transactions'])
transactions = pd.DataFrame(te_ary, columns=te.columns_)

frequent_items = apriori(transactions, min_support=0.005, use_colnames=True)

# Adicionar contagem de conjuntos de itens à lista de itens frequentes
frequent_items['length'] = frequent_items['itemsets'].apply(lambda x: len(x))

# Filtrar itens frequentes por comprimento e suporte mínimo
filtered_items = frequent_items[(frequent_items['length'] == 3) & (frequent_items['support'] >= 0.005)]

In [47]:
filtered_items

,support,itemsets,length
144,0.005894,"(butter, milk, packaged cheese)",3
145,0.005837,"(butter, milk, yogurt)",3
146,0.006061,"(butter, packaged cheese, yogurt)",3
147,0.005527,"(milk, cream, packaged cheese)",3
148,0.005793,"(yogurt, milk, cream)",3
149,0.005868,"(yogurt, packaged cheese, cream)",3
150,0.008014,"(milk, packaged cheese, eggs)",3
151,0.008557,"(yogurt, milk, eggs)",3
152,0.005310,"(packaged cheese, eggs, soy lactosefree)",3
153,0.008944,"(yogurt, packaged cheese, eggs)",3


import numpy as np
from google.colab import autoviz
df_2581732713416505518 = autoviz.get_registered_df('df_2581732713416505518')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_2581732713416505518, *['support'], **{})
chart

import numpy as np
from google.colab import autoviz
df_1918307214142781927 = autoviz.get_registered_df('df_1918307214142781927')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_1918307214142781927, *['support'], **{})
chart

import numpy as np
from google.colab import autoviz
df_7597766638061987499 = autoviz.get_registered_df('df_7597766638061987499')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_7597766638061987499, *['support'], **{})
chart

import numpy as np
from google.colab import autoviz
df_5181144984522962674 = autoviz.get_registered_df('df_5181144984522962674')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_5181144984522962674, *['support'], **{})
chart